In [17]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

In [18]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executer.memory", "4g")
    .getOrCreate()
)

In [19]:
sdf = spark.read.parquet("../data/curated/full_data_with_fraud/")
sdf.printSchema()
sdf.count()

root
 |-- user_id: long (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- SA2_code: integer (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- consumer_id: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- mean_total_income: integer (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- revenue_level: string (nullable = true)
 |-- take_rate: float (nullable = true)
 |-- is_fraud: integer (nullable = true)



13504142

In [20]:
# discard fraud transactions
sdf = sdf.filter(F.col('is_fraud')==0)
sdf.count()

13221647

In [21]:
# split the dataset 
train_sdf = sdf.filter((F.col('order_datetime') >= '2021-02-28') & (F.col('order_datetime') < '2021-08-28'))
label_sdf = sdf.filter((F.col('order_datetime') >= '2022-02-28') & (F.col('order_datetime') < '2022-08-28'))

In [22]:
train_sdf.count(), label_sdf.count()

(3457142, 4119273)

### Created Features
The following features are created by aggregating records from 2021-2 to 2021-8 for each merchant. They are used as features in the model.
1. Total number of consumers
2. Average transaction dollar value
3. Total number of transactions
4. Mean income of consumers
5. revenue level
6. BNPL revenue = take rate * total transaction
7. Number of distinct postcode
8. Tag (one hot encoding)

The following features are created by aggregating records from 2022-2 to 2022-8 for each merchant. These features are to be predicted by the model and are used as features in the final ranking system.
1. Total number of consumers
2. BNPL revenue
3. Total number of transactions

In [23]:
train_data = train_sdf.groupBy('merchant_abn')\
      .agg(
         F.countDistinct('consumer_id').alias('total_num_consumer'),
         F.mean('dollar_value').alias('avg_dollar_value'),
         F.countDistinct('order_id').alias('total_num_transaction'),
         F.mean('mean_total_income').alias('mean_income'),
         F.first('revenue_level').alias('revenue_level'),
         F.sum(F.col('dollar_value') * F.col('take_rate')).alias('total_revenue'),
         F.countDistinct('postcode').alias('total_num_postcode'),
         F.first('tags').alias('tag'),
      )
train_data

merchant_abn,total_num_consumer,avg_dollar_value,total_num_transaction,mean_income,revenue_level,total_revenue,total_num_postcode,tag
10023283211,808,209.02516040076958,823,62506.642770352366,e,30964.988492208053,719,furniture
10142254217,731,41.119659381592804,750,62693.954666666665,b,130143.71547230528,642,cable
10187291046,87,111.08408713922161,87,61060.0459770115,b,31795.59789319502,87,watch
10192359162,107,451.1432080236007,107,65637.3831775701,a,305563.80254357896,107,music
10206519221,2244,39.20921658203574,2370,62122.52278481013,a,589149.8606977111,1619,gift
10255988167,218,389.5552654520502,218,63146.619266055044,b,366867.58137013006,211,computer
10264435225,1238,114.10783402533238,1272,62006.311320754714,c,346896.95929006604,1018,watch
10279061213,130,308.35150039873355,130,63676.28461538462,a,228889.32027512783,127,computer
10323485998,2475,128.98933167858434,2621,62792.286913391836,a,2234715.708497144,1730,furniture
10342410215,238,378.0169713940928,238,61579.6512605042,a,570397.3822040077,227,computer


In [24]:
label = label_sdf.groupBy('merchant_abn')\
      .agg(
         F.countDistinct('consumer_id').alias('y_total_num_consumer'),
         F.sum(F.col('dollar_value') * F.col('take_rate')).alias('y_total_revenue'),
         F.countDistinct('order_id').alias('y_total_num_transaction')
      )

In [25]:
train_data.count(), label.count() 

(3953, 3789)

In [26]:
train_data = train_data.join(label, ["merchant_abn"], how="left") 
# use left join here since if no historical data is provided, we cannot predict the future value of a merchant
train_data

merchant_abn,total_num_consumer,avg_dollar_value,total_num_transaction,mean_income,revenue_level,total_revenue,total_num_postcode,tag,y_total_num_consumer,y_total_revenue,y_total_num_transaction
10023283211,808,209.0251604007696,823,62506.642770352366,e,30964.988492208053,719,furniture,980,38940.2914088986,1002
10142254217,731,41.119659381592804,750,62693.954666666665,b,130143.71547230528,642,cable,918,154467.92091896123,925
10187291046,87,111.08408713922161,87,61060.0459770115,b,31795.59789319502,87,watch,99,41683.211213258364,100
10192359162,107,451.1432080236007,107,65637.3831775701,a,305563.80254357896,107,music,107,290071.0062352741,107
10206519221,2244,39.209216582035744,2370,62122.52278481013,a,589149.8606977111,1619,gift,2662,666766.4067054288,2811
10255988167,218,389.55526545205026,218,63146.619266055044,b,366867.5813701301,211,computer,235,378005.14673149673,236
10264435225,1238,114.10783402533238,1272,62006.311320754714,c,346896.9592900661,1018,watch,1519,435003.6795629894,1566
10279061213,130,308.35150039873355,130,63676.28461538462,a,228889.32027512783,127,computer,161,273146.77071188006,161
10323485998,2475,128.98933167858425,2621,62792.286913391836,a,2234715.7084971434,1730,furniture,2974,2765582.918877613,3191
10342410215,238,378.01697139409276,238,61579.6512605042,a,570397.3822040078,227,computer,273,661666.9517852827,275


In [27]:
train_data.write.format('parquet').mode('overwrite').save("../data/curated/train_data")